# Защита персональных данных клиентов

Нужно защитить данные клиентов страховой компании «Хоть потоп». Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

<b> Описание данных </b>

Набор данных находится в файле: datasets/insurance.csv

- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Описание хода работы

1. Загрузить и изучить данные.
2. Ответить на вопрос и обосновать решение. 
     - Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
         - a. Изменится. Привести примеры матриц.
         - b. Не изменится. Указать, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
3. Предложить алгоритм преобразования данных для решения задачи. Обосновать, почему качество линейной регрессии не поменяется.
4. Запрограммировать этот алгоритм, применив матричные операции. Проверить, что качество линейной регрессии из sklearn не отличается до и после преобразования. Применить метрику R2.

## Настройки рабочей тетради

In [1]:
# Импорт библиотек

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import os

In [2]:
# Настройки библиотек

 
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)

In [3]:
# Путь к директории с данными

path = 'datasets/'

## Загрузка и предобработка данных

In [4]:
# Загрузка данных

data = pd.read_csv(os.path.join(path, 'insurance.csv'))

display(data)
data.info()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Пропусков нет. Для экономии памяти можно изменить тип данных во всех столбцах, кроме "Зарплата", на int8, а в столбце "зарплата" на int32

In [5]:
# Изменение типов

data['Зарплата'] = data['Зарплата'].astype(int)

for column in data.columns:
    data[column] = pd.to_numeric(data[column], downcast='integer')
    
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                5000 non-null   int8 
 1   Возраст            5000 non-null   int8 
 2   Зарплата           5000 non-null   int32
 3   Члены семьи        5000 non-null   int8 
 4   Страховые выплаты  5000 non-null   int8 
dtypes: int32(1), int8(4)
memory usage: 39.2 KB


In [6]:
display(data.describe())
display(data.corr())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.359400,1.194200,0.148000
std,0.500049,8.440807,9900.082063,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.002074,0.014910,-0.008991,0.010140
Возраст,0.002074,1.000000,-0.019093,-0.006692,0.651030
Зарплата,0.014910,-0.019093,1.000000,-0.030296,-0.014963
Члены семьи,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
Страховые выплаты,0.010140,0.651030,-0.014963,-0.036290,1.000000


Можно увидеть, что в таблице данных компании соблюден почти идеальный баланс между мужчинами и женщинами. Средний возраст застрахованного составляет около 31 года, при этом медиана составляет 30. Границы возраста застрахованных приводят к выводу, что компания не занимается страхованием лиц, чей возраст составляет менее 18 и старше 65 лет. Также интересно взглянуть на уровень заработной платы застрахованных - медиана составляет чуть более 40 000 рублей. В целом, в компании скорее представлен средний экономический класс, чем слишком бедные или слишком богатые граждане. 
    
Также можно посмотреть корреляцию. Возраст и количество страховых выплат клиенту за последние 5 лет имеют корреляцию 0.651. Что выглядит логично: чем старше застрахованное лицо, тем более вероятно он потребует страховку чаще, следовательно, несёт в себе больше рисков для предприятия.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$


**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** не изменится

**Обоснование:**

При умножении признаков на обратимую матрицу, предсказания не меняются. Докажем это.

Пусть R - обратимая матрица, с которой возможно произведение матрицы признаков X. 

Подставим в формуле вычисления предсказаний вместо X произведение XR и формулу для вычисления вектора весов линейной регрессии $a=Xw$
$$
a_1 = XRw = (XR)((XR)^T (XR))^{-1} (XR)^T y
$$
Раскроем транспонирование матриц
$$
a_1 = (XR)(R^TX^TXR)^{-1}R^TX^T y \qquad (1)
$$
Следовательно $(R^TX^TXR)^{-1}=(R^{-1}(X^TX)^{-1}(R^T)^{-1})$. Подставляем в (1):
$$
a_1 = XRR^{-1}(X^TX)^{-1}(R^T)^{-1}R^TX^T y = XE(X^TX)^{-1}EX^Ty = X(X^TX)^{-1}X^Ty = a
$$


## Алгоритм преобразования

**Алгоритм**

Исходя из предыдущего пункта можно понять, что качество модели не ухудшится, если умножить матрицу признаков на обратимую матрицу. Значит, для защиты персональных данных можно умножить матрицу признаков на случайно сгенерированную обратимую матрицу. Для этого нужно:

- сгенерировать случайную матрицу, на которую можно будет умножить матрицу признаков
- проверить сгенерированную матрицу на обратимость
- преобразовать матрицу признаков умножением её на сгенерированную матрицу

**Обоснование**

Чтобы на случайно сгенерированную матрицу можно было умножить матрицу признаков и при этом сохранить размерность, размер такой сгенерированной матрицы необходимо задать $nxn$, где n - количество признаков (столбцов в матрице в признаков).

Чтобы проверить случайно сгенерированную матрицу на обратимость, необходимо проверить, чтобы её определитель не равнялся нулю.

## Проверка алгоритма

In [7]:
# Реализация преобразования данных в виде функции

def crypt_features(features):
    '''Функция преобразования матрицы признаков'''
    
    np.random.seed(123456)
    
    n = features.shape[1]
    reversible_matrix = np.random.randint(1, 100, size=(n,n))
    
    op = np.linalg.det(reversible_matrix)
    while op == 0:
        np.random.seed(123456)
        reversible_matrix = np.random.randint(1, 100, size=(n,n))
        op = np.linalg.det(reversible_matrix)
        
    crypted_features = np.dot(features, reversible_matrix)
    
    return crypted_features

In [8]:
# Нахождение качества линейной регрессии ДО преобразования данных


    # Разделение данных на выборки
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']
features_train, features_test, target_train, target_test = train_test_split(features, target, 
                                                                            test_size=0.25, random_state=123456)


model = LinearRegression()
model.fit(features_train, target_train)

predictions = model.predict(features_test)
R2_original_data = r2_score(target_test, predictions)

print("До преобразования данных метрика R2 =", R2_original_data)

До преобразования данных метрика R2 = 0.41921160241971167


In [9]:
# Нахождение качества линейной регрессии ПОСЛЕ преобразования данных


crypted_features = crypt_features(features)

    # Разделение данных на выборки
features_train, features_test, target_train, target_test = train_test_split(crypted_features, target, 
                                                                            test_size=0.25, random_state=123456)


model = LinearRegression()
model.fit(features_train, target_train)

predictions = model.predict(features_test)
R2_crypted_data = r2_score(target_test, predictions)

print("После преобразования данных R2 =", R2_crypted_data)

После преобразования данных R2 = 0.4192116024197241


### Вывод

Предложенный алгоритм реализован. Значения метрики R2 до и после преобразования данных почти идентичны, различия наблюдаются только после 13 знаков после запятой.